# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jan 4 2023 10:18AM,254257,19,CLI00442.1,"CLINUVEL, Inc.",Released
1,Jan 4 2023 10:03AM,254256,10,CLI00443.1,"CLINUVEL, Inc.",Released
2,Jan 4 2023 10:03AM,254256,10,CLI00444.1,"CLINUVEL, Inc.",Released
3,Jan 4 2023 9:59AM,254255,19,TR-RC-01062023,"GCH Granules USA, Inc.",Released
4,Jan 4 2023 9:51AM,254254,16,EMERSA-8988983,"Emersa Waterbox, LLC",Released
5,Jan 4 2023 9:39AM,254252,16,SHL-8988962,"SHL Pharma, LLC",Released
6,Jan 4 2023 9:36AM,254251,10,CTF0011569,"Citieffe, Inc.",Released
7,Jan 4 2023 9:36AM,254251,10,CTF0011568,"Citieffe, Inc.",Released
8,Jan 4 2023 9:30AM,254248,10,PRF-45914,Bio-PRF,Released
9,Jan 4 2023 9:30AM,254248,10,PRF-45918,Bio-PRF,Executing


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
34,254252,Released,1
35,254254,Released,1
36,254255,Released,1
37,254256,Released,2
38,254257,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
254252,NaN,NaN,1.0
254254,NaN,NaN,1.0
254255,NaN,NaN,1.0
254256,NaN,NaN,2.0
254257,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
254141,0.0,0.0,54.0
254143,0.0,0.0,42.0
254146,0.0,0.0,31.0
254150,0.0,0.0,28.0
254153,0.0,0.0,4.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
254141,0,0,54
254143,0,0,42
254146,0,0,31
254150,0,0,28
254153,0,0,4


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,254141,0,0,54
1,254143,0,0,42
2,254146,0,0,31
3,254150,0,0,28
4,254153,0,0,4


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,254141,,,54
1,254143,,,42
2,254146,,,31
3,254150,,,28
4,254153,,,4


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jan 4 2023 10:18AM,254257,19,"CLINUVEL, Inc."
1,Jan 4 2023 10:03AM,254256,10,"CLINUVEL, Inc."
3,Jan 4 2023 9:59AM,254255,19,"GCH Granules USA, Inc."
4,Jan 4 2023 9:51AM,254254,16,"Emersa Waterbox, LLC"
5,Jan 4 2023 9:39AM,254252,16,"SHL Pharma, LLC"
6,Jan 4 2023 9:36AM,254251,10,"Citieffe, Inc."
8,Jan 4 2023 9:30AM,254248,10,Bio-PRF
22,Jan 4 2023 9:23AM,254249,10,"SD Head USA, LLC"
23,Jan 4 2023 8:51AM,254241,10,"Nextsource Biotechnology, LLC"
24,Jan 4 2023 8:30AM,254196,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jan 4 2023 10:18AM,254257,19,"CLINUVEL, Inc.",,,1
1,Jan 4 2023 10:03AM,254256,10,"CLINUVEL, Inc.",,,2
2,Jan 4 2023 9:59AM,254255,19,"GCH Granules USA, Inc.",,,1
3,Jan 4 2023 9:51AM,254254,16,"Emersa Waterbox, LLC",,,1
4,Jan 4 2023 9:39AM,254252,16,"SHL Pharma, LLC",,,1
5,Jan 4 2023 9:36AM,254251,10,"Citieffe, Inc.",,,2
6,Jan 4 2023 9:30AM,254248,10,Bio-PRF,,1,13
7,Jan 4 2023 9:23AM,254249,10,"SD Head USA, LLC",,,1
8,Jan 4 2023 8:51AM,254241,10,"Nextsource Biotechnology, LLC",,,1
9,Jan 4 2023 8:30AM,254196,10,ISDIN Corporation,,,17


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 4 2023 10:18AM,254257,19,"CLINUVEL, Inc.",1,,
1,Jan 4 2023 10:03AM,254256,10,"CLINUVEL, Inc.",2,,
2,Jan 4 2023 9:59AM,254255,19,"GCH Granules USA, Inc.",1,,
3,Jan 4 2023 9:51AM,254254,16,"Emersa Waterbox, LLC",1,,
4,Jan 4 2023 9:39AM,254252,16,"SHL Pharma, LLC",1,,
5,Jan 4 2023 9:36AM,254251,10,"Citieffe, Inc.",2,,
6,Jan 4 2023 9:30AM,254248,10,Bio-PRF,13,1,
7,Jan 4 2023 9:23AM,254249,10,"SD Head USA, LLC",1,,
8,Jan 4 2023 8:51AM,254241,10,"Nextsource Biotechnology, LLC",1,,
9,Jan 4 2023 8:30AM,254196,10,ISDIN Corporation,17,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 4 2023 10:18AM,254257,19,"CLINUVEL, Inc.",1,,
1,Jan 4 2023 10:03AM,254256,10,"CLINUVEL, Inc.",2,,
2,Jan 4 2023 9:59AM,254255,19,"GCH Granules USA, Inc.",1,,
3,Jan 4 2023 9:51AM,254254,16,"Emersa Waterbox, LLC",1,,
4,Jan 4 2023 9:39AM,254252,16,"SHL Pharma, LLC",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 4 2023 10:18AM,254257,19,"CLINUVEL, Inc.",1.0,NaN,NaN
1,Jan 4 2023 10:03AM,254256,10,"CLINUVEL, Inc.",2.0,NaN,NaN
2,Jan 4 2023 9:59AM,254255,19,"GCH Granules USA, Inc.",1.0,NaN,NaN
3,Jan 4 2023 9:51AM,254254,16,"Emersa Waterbox, LLC",1.0,NaN,NaN
4,Jan 4 2023 9:39AM,254252,16,"SHL Pharma, LLC",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 4 2023 10:18AM,254257,19,"CLINUVEL, Inc.",1.0,0.0,0.0
1,Jan 4 2023 10:03AM,254256,10,"CLINUVEL, Inc.",2.0,0.0,0.0
2,Jan 4 2023 9:59AM,254255,19,"GCH Granules USA, Inc.",1.0,0.0,0.0
3,Jan 4 2023 9:51AM,254254,16,"Emersa Waterbox, LLC",1.0,0.0,0.0
4,Jan 4 2023 9:39AM,254252,16,"SHL Pharma, LLC",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3558710,204.0,2.0,1.0
15,1016753,157.0,8.0,59.0
16,508506,2.0,0.0,0.0
19,2287905,69.0,8.0,0.0
20,254237,0.0,1.0,0.0
21,508419,1.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3558710,204.0,2.0,1.0
1,15,1016753,157.0,8.0,59.0
2,16,508506,2.0,0.0,0.0
3,19,2287905,69.0,8.0,0.0
4,20,254237,0.0,1.0,0.0
5,21,508419,1.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,204.0,2.0,1.0
1,15,157.0,8.0,59.0
2,16,2.0,0.0,0.0
3,19,69.0,8.0,0.0
4,20,0.0,1.0,0.0
5,21,1.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,204.0
1,15,Released,157.0
2,16,Released,2.0
3,19,Released,69.0
4,20,Released,0.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,19,20,21
Status,,,,,,
Completed,1.0,59.0,0.0,0.0,0.0,0.0
Executing,2.0,8.0,0.0,8.0,1.0,1.0
Released,204.0,157.0,2.0,69.0,0.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,19,20,21
0,Completed,1.0,59.0,0.0,0.0,0.0,0.0
1,Executing,2.0,8.0,0.0,8.0,1.0,1.0
2,Released,204.0,157.0,2.0,69.0,0.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,19,20,21
0,Completed,1.0,59.0,0.0,0.0,0.0,0.0
1,Executing,2.0,8.0,0.0,8.0,1.0,1.0
2,Released,204.0,157.0,2.0,69.0,0.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()